In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matchzoo as mz
from tqdm.auto import tqdm
import os
from matchzoo.data_generator import DataGeneratorBuilder
import shutil
# preprocessor = mz.load_preprocessor("processor.mz")
mz.__version__

Using TensorFlow backend.


'2.1.0'

In [29]:
from keras.utils import multi_gpu_model

In [30]:
train_data = mz.data_pack.load_data_pack("train.processed.datapack/")
dev_data = mz.data_pack.load_data_pack("dev.processed.datapack/")
test_data = mz.data_pack.load_data_pack("test.processed.datapack/")

In [101]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3,4,5,6,7"  # specify which GPU(s) to be used

class ANMMConfig():
    # preprocessor = mz.preprocessors.ANMMPreprocessor()
    optimizer = 'SGD'
    model = mz.models.ANMM
    generator_flag = 1
    name = 'anmm'
    num_dup = 1
    num_neg = 4
    num_layers = 2
    shuffle = True
    ###### training config ######
    batch_size = 128
    epoch = 3
    ##### save config #####
    parent_path = '/ssd2/arthur/matchzoo/saved_results'
    model_parent_path = '/ssd2/arthur/matchzoo/ANMM'
    model_save_path = os.path.join(model_parent_path,name)
    parameter_set = []
    parameter_set1 = []
    parameter_name = ['dropout_rate','num_layers']


In [107]:
class BasicModel(object):
    def __init__(self, config):
        self.config = config

    def mkdir(self):
        folder = os.path.exists(self.config.model_save_path)
        if not folder:
            os.makedirs(self.config.model_save_path)
            print("---  new folder...  ---")
            print("---  new folder...  ---")
        else:
            print("---  There is this folder!  ---")

    def parameter_get(self):
        return self.config.parameter_set,self.config.parameter_name
    def name(self):
        return self.config.name
    def get_path(self):
        return self.config.model_save_path,self.config.model_parent_path

    def model_delete(self):
        trash_dir = self.config.model_parent_path
        try:
            shutil.rmtree(trash_dir)
        except OSError as e:
            print(f'Error: {trash_dir} : {e.strerror}')
        os.mkdir(trash_dir)

    def auto_prepare(self,train_pack,valid_pack,test_pack):
        model = self.config.model()
        task = mz.tasks.Ranking(metrics=['mean_average_precision'])
        model.params['task'] = task
        model_ok, train_ok, preprocesor_ok = mz.auto.prepare(task=task, model_class=type(model), data_pack=train_pack)
        test_ok = preprocesor_ok.transform(test_pack, verbose=0)
        valid_ok = preprocesor_ok.transform(valid_pack, verbose=0)
        return model_ok, train_ok, test_ok, valid_ok
    def get_lr(self):
        return self.config.learning_rate


In [108]:
config = ANMMConfig()
model = BasicModel(config)
model_name = model.name()
model.model_delete()

In [109]:
task = mz.tasks.Ranking(metrics=['mean_average_precision', 'ndcg@100'])
prepared = mz.auto.Preparer(task)
model_class = mz.models.ANMM


In [110]:
model = model_class()
model.params['task'] = task
model.params.update(preprocessor.context)
model.guess_and_fill_missing_params()
model.build()
model._backend = multi_gpu_model(model._backend, gpus=5)
model.compile()
data_gen = DataGeneratorBuilder(mode="point", batch_size=128, shuffle=True).build(train_data)
callback = mz.engine.callbacks.EvaluateAllMetrics(model, *dev_data.unpack(), batch_size=128,verbose=0,model_save_path="/ssd2/arthur/matchzoo/ANMM/")

Parameter "input_shapes" set to [(30,), (30,)].
Parameter "embedding_output_dim" set to 300.


In [ ]:
history = model.fit_generator(data_gen, epochs=3, validation_freq=1, callbacks=[callback, EarlyStopping(monitor='val_loss', patience=1)])

Epoch 1/3
 4601/31591 [===>..........................] - ETA: 54:08 - loss: 0.0834

In [ ]:
import subprocess
split="test"

if split=="test":
    triples_file = "/ssd2/arthur/TREC2019/data/triples-tokenized/cut-{}.top100".format(split)
else:
    triples_file = "/ssd2/arthur/TREC2019/data/triples-tokenized/cut-{}.10neg.fix".format(split)
assert os.path.isfile(triples_file)
expected_len = int(subprocess.check_output("wc -l {}".format(triples_file).split()).decode().split()[0])
all_docs = []
relationships = []
processed_docs = set()
for line in tqdm(open(triples_file), total=expected_len):
    _id, doc, label = line.strip().split("\t")
    label = int(label)
    q_id, d_id = _id.split("-")
    if label == '1' and split!="test":
        relationships.append([q_id, d_id, label])
    elif split == "test":
        relationships.append([q_id, d_id, label])
    if d_id in processed_docs:
        continue
    doc = " ".join(eval(doc)).replace(" ##", "").split("[SEP]")[1]
    all_docs.append([d_id, doc])



In [4]:
for line in tqdm(open(triples_file), total=expected_len):
    _id, doc, label = line.strip().split("\t")
    label = int(label)
    relationships.append([q_id, d_id, label])

In [5]:
!rm /ssd2/arthur/TREC2019/data/triples-tokenized/$split.datapack/data.dill
relation_df = pd.DataFrame(relationships)
relation_df.columns = ["id_left", "id_right", "label"]
left = pd.DataFrame(queries)
left.columns = ['id_left', 'text_left']
left.set_index('id_left', inplace=True)
right = pd.DataFrame(all_docs)
right.columns = ['id_right', 'text_right']
right.set_index('id_right', inplace=True)
dp = mz.DataPack(relation=relation_df, left=left, right=right)
dp.save("/ssd2/arthur/TREC2019/data/triples-tokenized/{}.datapack".format(split))

rm: cannot remove '/ssd2/arthur/TREC2019/data/triples-tokenized/.datapack/data.dill': No such file or directory


NameError: name 'queries' is not defined

In [ ]:
!rm /ssd2/arthur/TREC2019/data/triples-tokenized/test.datapack/data.dill

In [ ]:
relation_df = pd.DataFrame(relationships)
relation_df.columns = ["id_left", "id_right", "label"]
left = pd.DataFrame(queries)
left.columns = ['id_left', 'text_left']
left.set_index('id_left', inplace=True)
right = pd.DataFrame(all_docs)
right.columns = ['id_right', 'text_right']
right.set_index('id_right', inplace=True)
# dp = mz.DataPack(relation=relation_df, left=left, right=right)
# dp.save("/ssd2/arthur/TREC2019/data/triples-tokenized/{}.datapack".format(split))

In [ ]:
from tqdm import tqdm_notebook as tqdm
import matchzoo as mz

In [ ]:
task = mz.tasks.Ranking()
model = mz.models.ANMM()

In [ ]:
preprocessor = mz.load_preprocessor("processor.mz")

In [ ]:
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] =task
model.guess_and_fill_missing_params()

In [ ]:
model.build()
model.compile()

In [ ]:
train_generator = mz.DataGenerator(train_data, batch_size=64, shuffle=True)

In [ ]:
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=valid_x, y=valid_y, batch_size=128)

In [ ]:
history = model.fit_generator(train_generator, epochs=3, callbacks=[evaluate], workers=20, use_multiprocessing=True)

In [3]:
from matchzoo import DataPack
from matchzoo.engine.base_preprocessor import BasePreprocessor
from matchzoo.preprocessors.build_vocab_unit import build_vocab_unit
from matchzoo.preprocessors.build_unit_from_data_pack import build_unit_from_data_pack
from matchzoo.preprocessors.chain_transform import chain_transform
from matchzoo.preprocessors import units

In [15]:
class MyPreprocessor(BasePreprocessor):
    def __init__(self, filter_low_freq=1000):
        super().__init__()
        self._left_fixedlength_unit = units.FixedLength(30, pad_mode='post')
        self._right_fixedlength_unit = units.FixedLength(30, pad_mode='post')
        self._filter_unit = units.FrequencyFilter(low=filter_low_freq, mode='df')

    def fit(self, data_pack: DataPack, verbose=1):
#         data_pack = data_pack.apply_on_text(chain_transform(self.units), verbose=verbose)
        fitted_filter_unit = build_unit_from_data_pack(self._filter_unit, data_pack, flatten=False, mode='right', verbose=verbose)
        data_pack = data_pack.apply_on_text(fitted_filter_unit.transform, mode='right', verbose=verbose)
        self._context['filter_unit'] = fitted_filter_unit
        vocab_unit = build_vocab_unit(data_pack, verbose=verbose)
        self._context['vocab_unit'] = vocab_unit
        vocab_size = len(vocab_unit.state['term_index'])
        self._context['vocab_size'] = vocab_size
        self._context['embedding_input_dim'] = vocab_size
        return self

    def transform(self, data_pack, verbose=1):
        data_pack = data_pack.copy()
        data_pack.apply_on_text(self._context['filter_unit'].transform, mode="right", inplace=True, verbose=verbose)
        data_pack.apply_on_text(self._context['vocab_unit'].transform, mode='both', inplace=True, verbose=verbose)
        data_pack.apply_on_text(self._left_fixedlength_unit.transform, mode='left', inplace=True, verbose=verbose)
        data_pack.apply_on_text(self._right_fixedlength_unit.transform, mode='right', inplace=True, verbose=verbose)

        data_pack.append_text_length(inplace=True, verbose=verbose)
        return data_pack
preprocessor = MyPreprocessor()

In [19]:
test_data = mz.data_pack.load_data_pack("test.processed.datapack/")
train_data = mz.data_pack.load_data_pack("train.processed.datapack/")

In [24]:
train_processed = preprocessor.fit_transform(train_data)




Processing text_right with append:   0%|          | 0/4043622 [00:00<?, ?it/s]


Processing text_right with append:   1%|          | 35222/4043622 [00:00<00:11, 352219.50it/s]


Processing text_right with append:   4%|▍         | 157849/4043622 [00:00<00:08, 448020.17it/s]


Processing text_right with append:   7%|▋         | 285609/4043622 [00:00<00:06, 556406.74it/s]


Processing text_right with append:  10%|█         | 419285/4043622 [00:00<00:05, 674538.07it/s]


Processing text_right with append:  14%|█▍        | 562147/4043622 [00:00<00:04, 801448.14it/s]


Processing text_right with append:  18%|█▊        | 709652/4043622 [00:00<00:03, 928675.68it/s]


Processing text_right with append:  21%|██▏       | 859705/4043622 [00:00<00:03, 1048556.94it/s]


Processing text_right with append:  25%|██▍       | 1007304/4043622 [00:00<00:02, 1148318.75it/s]


Processing text_right with append:  28%|██▊       | 1148602/4043622 [00:00<00:02, 1216685.80it/s]


Processing text_right with app

Building FrequencyFilter from a datapack.:  39%|███▉      | 1567544/4043622 [00:11<00:17, 142520.13it/s]


Building FrequencyFilter from a datapack.:  39%|███▉      | 1581800/4043622 [00:11<00:17, 142234.11it/s]


Building FrequencyFilter from a datapack.:  39%|███▉      | 1596116/4043622 [00:11<00:17, 142508.32it/s]


Building FrequencyFilter from a datapack.:  40%|███▉      | 1610369/4043622 [00:11<00:17, 139867.36it/s]


Building FrequencyFilter from a datapack.:  40%|████      | 1624652/4043622 [00:11<00:17, 140742.51it/s]


Building FrequencyFilter from a datapack.:  41%|████      | 1638981/4043622 [00:11<00:16, 141496.64it/s]


Building FrequencyFilter from a datapack.:  41%|████      | 1653365/4043622 [00:11<00:16, 142189.82it/s]


Building FrequencyFilter from a datapack.:  41%|████      | 1667591/4043622 [00:11<00:16, 141266.59it/s]


Building FrequencyFilter from a datapack.:  42%|████▏     | 1681724/4043622 [00:12<00:17, 138424.21it/s]


Building FrequencyFilter from a datap

Building FrequencyFilter from a datapack.:  84%|████████▍ | 3397495/4043622 [00:25<00:04, 135388.63it/s]


Building FrequencyFilter from a datapack.:  84%|████████▍ | 3411037/4043622 [00:25<00:04, 134439.91it/s]


Building FrequencyFilter from a datapack.:  85%|████████▍ | 3424485/4043622 [00:25<00:04, 133920.94it/s]


Building FrequencyFilter from a datapack.:  85%|████████▌ | 3437880/4043622 [00:25<00:04, 133739.65it/s]


Building FrequencyFilter from a datapack.:  85%|████████▌ | 3451256/4043622 [00:26<00:04, 132621.75it/s]


Building FrequencyFilter from a datapack.:  86%|████████▌ | 3464522/4043622 [00:26<00:04, 132532.32it/s]


Building FrequencyFilter from a datapack.:  86%|████████▌ | 3477787/4043622 [00:26<00:04, 132565.18it/s]


Building FrequencyFilter from a datapack.:  86%|████████▋ | 3491072/4043622 [00:26<00:04, 132648.54it/s]


Building FrequencyFilter from a datapack.:  87%|████████▋ | 3504339/4043622 [00:26<00:04, 132247.00it/s]


Building FrequencyFilter from a datap

Processing text_right with transform:  39%|███▉      | 1588726/4043622 [00:16<00:14, 166530.00it/s]


Processing text_right with transform:  40%|███▉      | 1605561/4043622 [00:16<00:14, 167070.36it/s]


Processing text_right with transform:  40%|████      | 1622273/4043622 [00:16<00:14, 166549.70it/s]


Processing text_right with transform:  41%|████      | 1638932/4043622 [00:16<00:14, 163290.85it/s]


Processing text_right with transform:  41%|████      | 1655278/4043622 [00:17<00:14, 159718.50it/s]


Processing text_right with transform:  41%|████▏     | 1671279/4043622 [00:17<00:15, 157815.11it/s]


Processing text_right with transform:  42%|████▏     | 1688384/4043622 [00:17<00:14, 161565.35it/s]


Processing text_right with transform:  42%|████▏     | 1705567/4043622 [00:17<00:14, 164510.63it/s]


Processing text_right with transform:  43%|████▎     | 1722678/4043622 [00:17<00:13, 166435.14it/s]


Processing text_right with transform:  43%|████▎     | 1739752/4043622 [00:17<00:1

Processing text_right with transform: 100%|█████████▉| 4026283/4043622 [00:36<00:00, 156157.93it/s]


Processing text_right with transform: 100%|██████████| 4043622/4043622 [00:36<00:00, 110740.31it/s]



Processing text_left with extend:   0%|          | 0/367013 [00:00<?, ?it/s]


Processing text_left with extend:  13%|█▎        | 47672/367013 [00:00<00:00, 476717.04it/s]


Processing text_left with extend:  34%|███▎      | 123145/367013 [00:00<00:00, 535943.02it/s]


Processing text_left with extend:  55%|█████▍    | 200931/367013 [00:00<00:00, 591092.34it/s]


Processing text_left with extend:  75%|███████▍  | 274988/367013 [00:00<00:00, 629190.40it/s]


Processing text_left with extend: 100%|██████████| 367013/367013 [00:00<00:00, 694012.79it/s]



Processing text_right with extend:   0%|          | 0/4043622 [00:00<?, ?it/s]


Processing text_right with extend:   1%|          | 43911/4043622 [00:00<00:09, 439105.18it/s]


Processing text_right with extend:   3%|▎         | 136634

...

Processing text_right with transform:   2%|▏         | 99330/4043622 [00:00<00:30, 128297.93it/s]


Processing text_right with transform:   3%|▎         | 115270/4043622 [00:00<00:28, 136274.86it/s]


Processing text_right with transform:   3%|▎         | 131068/4043622 [00:00<00:27, 142131.26it/s]


Processing text_right with transform:   4%|▎         | 146999/4043622 [00:01<00:26, 146880.29it/s]


Processing text_right with transform:   4%|▍         | 162826/4043622 [00:01<00:25, 150121.22it/s]


Processing text_right with transform:   4%|▍         | 178327/4043622 [00:01<00:25, 151554.47it/s]


Processing text_right with transform:   5%|▍         | 193805/4043622 [00:01<00:25, 152507.53it/s]


Processing text_right with transform:   5%|▌         | 209270/4043622 [00:01<00:26, 147138.96it/s]


Processing text_right with transform:   6%|▌         | 224175/4043622 [00:01<00:26, 144363.13it/s]


Processing text_right with transform:   6%|▌         | 238757/4043622 [00:01<00:26, 144062.6

Processing text_right with transform:  56%|█████▌    | 2250782/4043622 [00:21<00:12, 139549.75it/s]


Processing text_right with transform:  56%|█████▌    | 2266674/4043622 [00:21<00:12, 144844.16it/s]


Processing text_right with transform:  56%|█████▋    | 2282229/4043622 [00:22<00:11, 147896.65it/s]


Processing text_right with transform:  57%|█████▋    | 2297960/4043622 [00:22<00:11, 150599.21it/s]


Processing text_right with transform:  57%|█████▋    | 2313089/4043622 [00:22<00:11, 145213.32it/s]


Processing text_right with transform:  58%|█████▊    | 2327701/4043622 [00:22<00:12, 141720.38it/s]


Processing text_right with transform:  58%|█████▊    | 2341955/4043622 [00:22<00:12, 141209.85it/s]


Processing text_right with transform:  58%|█████▊    | 2356134/4043622 [00:22<00:12, 138124.11it/s]


Processing text_right with transform:  59%|█████▊    | 2370002/4043622 [00:22<00:12, 137701.26it/s]


Processing text_right with transform:  59%|█████▉    | 2383941/4043622 [00:22<00:1

Processing text_left with transform:  79%|███████▉  | 290161/367013 [00:02<00:00, 107220.31it/s]


Processing text_left with transform:  82%|████████▏ | 301034/367013 [00:03<00:00, 107667.30it/s]


Processing text_left with transform:  85%|████████▌ | 311962/367013 [00:03<00:00, 107780.84it/s]


Processing text_left with transform:  88%|████████▊ | 322743/367013 [00:03<00:00, 107265.32it/s]


Processing text_left with transform:  91%|█████████ | 333530/367013 [00:03<00:00, 107443.90it/s]


Processing text_left with transform:  94%|█████████▍| 344423/367013 [00:03<00:00, 107883.34it/s]


Processing text_left with transform:  97%|█████████▋| 355214/367013 [00:03<00:00, 106492.68it/s]


Processing text_left with transform: 100%|██████████| 367013/367013 [00:03<00:00, 100924.18it/s]



Processing text_right with transform:   0%|          | 0/4043622 [00:00<?, ?it/s]


Processing text_right with transform:   0%|          | 4956/4043622 [00:04<1:05:37, 1025.82it/s]


Processing text_right wi

Processing text_right with transform:  36%|███▌      | 1458746/4043622 [00:19<00:24, 106150.95it/s]


Processing text_right with transform:  36%|███▋      | 1469365/4043622 [00:19<00:24, 106050.76it/s]


Processing text_right with transform:  37%|███▋      | 1479973/4043622 [00:19<00:24, 104772.28it/s]


Processing text_right with transform:  37%|███▋      | 1490456/4043622 [00:19<00:24, 104752.10it/s]


Processing text_right with transform:  37%|███▋      | 1500935/4043622 [00:19<00:24, 104216.05it/s]


Processing text_right with transform:  37%|███▋      | 1511615/4043622 [00:19<00:24, 104977.45it/s]


Processing text_right with transform:  38%|███▊      | 1522117/4043622 [00:19<00:24, 104768.94it/s]


Processing text_right with transform:  38%|███▊      | 1532597/4043622 [00:19<00:24, 103859.56it/s]


Processing text_right with transform:  38%|███▊      | 1542987/4043622 [00:19<00:24, 103220.28it/s]


Processing text_right with transform:  38%|███▊      | 1553313/4043622 [00:19<00:2

Processing text_right with transform:  72%|███████▏  | 2930823/4043622 [00:40<00:11, 100492.22it/s]


Processing text_right with transform:  73%|███████▎  | 2940873/4043622 [00:40<00:11, 100051.71it/s]


Processing text_right with transform:  73%|███████▎  | 2950880/4043622 [00:40<00:10, 99638.65it/s] 


Processing text_right with transform:  73%|███████▎  | 2960845/4043622 [00:40<00:10, 99500.26it/s]


Processing text_right with transform:  73%|███████▎  | 2970824/4043622 [00:40<00:10, 99584.62it/s]


Processing text_right with transform:  74%|███████▎  | 2980784/4043622 [00:40<00:10, 99347.47it/s]


Processing text_right with transform:  74%|███████▍  | 2990769/4043622 [00:40<00:10, 99495.03it/s]


Processing text_right with transform:  74%|███████▍  | 3000719/4043622 [00:40<00:12, 82779.24it/s]


Processing text_right with transform:  74%|███████▍  | 3009468/4043622 [00:41<00:12, 82942.82it/s]


Processing text_right with transform:  75%|███████▍  | 3018093/4043622 [00:41<00:12, 833

In [25]:
train_processed.save("train.processed.datapack")
test_processed = preprocessor.transform(test_data)
test_processed.save("test.processed.datapack")
dev_data = mz.data_pack.load_data_pack("dev.processed.datapack/")
dev_processed = preprocessor.transform(dev_data)
dev_processed.save("dev.processed.datapack")




Processing text_right with transform:   0%|          | 0/155800 [00:00<?, ?it/s]


Processing text_right with transform:   5%|▍         | 7616/155800 [00:00<00:01, 76153.54it/s]


Processing text_right with transform:  12%|█▏        | 18688/155800 [00:00<00:01, 84022.58it/s]


Processing text_right with transform:  20%|█▉        | 30547/155800 [00:00<00:01, 92072.79it/s]


Processing text_right with transform:  28%|██▊       | 42928/155800 [00:00<00:01, 99742.97it/s]


Processing text_right with transform:  35%|███▌      | 54802/155800 [00:00<00:00, 104769.89it/s]


Processing text_right with transform:  43%|████▎     | 67305/155800 [00:00<00:00, 110122.95it/s]


Processing text_right with transform:  51%|█████     | 79607/155800 [00:00<00:00, 113698.18it/s]


Processing text_right with transform:  59%|█████▉    | 92218/155800 [00:00<00:00, 117156.57it/s]


Processing text_right with transform:  68%|██████▊   | 106528/155800 [00:00<00:00, 123894.56it/s]


Processing text_right with 

FileExistsError: dev.processed.datapack/data.dill already exist, fail to save

In [39]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fa61f2ec898>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fa494793be0>,
 'vocab_size': 23463,
 'embedding_input_dim': 23463}

In [73]:
# model.save("/ssd2/arthur/matchzoo/ANMM/final")
_model = model

In [50]:
labels = model.predict(test_data.unpack()[0])

In [57]:
import pickle
pickle.dump(labels, open("anmm_scores.pkl", 'wb'))


In [71]:
ql_run_file = "/ssd2/arthur/TREC2019/data/runs/indri_test.run"
out_run_file = os.path.join("/ssd2/arthur/TREC2019/data/runs/test_anmm.run")
topic_results = []
last_topic = -1
runs_format = "{} Q0 {} {} {} ANMM\n" #topic_id, doc_id, ranking, score
with open(ql_run_file) as inf, open(out_run_file, 'w') as outf:
    for counter, (example, score) in enumerate(zip(inf, labels)):
        topic_id, _, doc_id, _, _, _ = example.split()
        if topic_id != last_topic and len(topic_results) > 0:
            topic_results.sort(key=lambda x:x['score'], reverse=True)
            for rank, topic in enumerate(topic_results):
                outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))
            topic_results = []
        topic_results.append({'topic_id':topic_id, 'doc_id':doc_id, 'score':score})
        last_topic = topic_id
    topic_results.sort(key=lambda x:x['score'], reverse=True)
    for rank, topic in enumerate(topic_results):
        outf.write(runs_format.format(topic['topic_id'], topic['doc_id'], rank, topic['score']))